# Worksheet 08

Name: Grace Van Sciver 
UID: U05141982

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [11]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[4.533339848232871, 5.544750001848204, 3.755015086666395, 4.797619041380229, 4.416708696894397, 7.057658106142439, 4.849106480214305, 3.0617015000551646, 6.311341892466097, 4.536603525045378]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [12]:
s2 = np.random.normal(8, 1, 10).tolist()
print(s2)

[8.01392805722326, 7.004309875326804, 9.088985936871264, 10.547640543473179, 8.714949306744996, 8.13470102372501, 9.78287866474352, 8.041722539493131, 9.432593980640974, 8.772126951503878]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [13]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = s1.pop()
        data.append(p1)
    else:
        p2 = s2.pop()
        data.append(p2)
print(data)

[8.772126951503878, 4.536603525045378, 6.311341892466097, 9.432593980640974, 3.0617015000551646, 4.849106480214305, 8.041722539493131, 9.78287866474352, 8.13470102372501, 7.057658106142439]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

K (num of mixture components) = 2
mu (means of gaussian distributions): mu1 = 5, mu2 = 8

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [14]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(s2)

prob_s = [ len(s1) / (len(s1) + len(s2)) , len(s2) / (len(s1) + len(s2)) ]
mean = [ sum(s1)/len(s1) , sum(s2)/len(s2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1) , sum(map(lambda x : (x - mean[1])**2, s2)) / len(s2) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[4.536603525045378, 6.311341892466097, 3.0617015000551646, 4.849106480214305]
[8.772126951503878, 9.432593980640974, 8.041722539493131, 9.78287866474352, 8.13470102372501, 7.057658106142439]
P(S_1) = 0.4,  P(S_2) = 0.6
mean_1 = 4.689688349445236,  mean_2 = 8.536946877708159
var_1 = 1.3322376248146404,  var_2 = 0.8341972599891996


/Users/gracevansciver/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Mean1 = 4.378       True Mean1 = 5
Mean2 = 6.984       True Mean2 = 8
Var1 = 0.483        True Var1 = 1
Var2 = 1.123        True Var2 = 1
P(S1) = 0.7         True P(S1) = 0.5
P(S2) = 0.3         True P(S2) = 0.5

The answers are relatively close to the true values with the variances differing the most. 

e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [15]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append( pdf_i[0] * prob_s[0] / prob_x )
    prob_s1_x.append( pdf_i[1] * prob_s[1] / prob_x )  

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


point =  8.772126951503878
probability of observing that point if it came from cluster 0 =  0.002736952261583532
probability of observing that point if it came from cluster 1 =  0.45960232694279035
point =  4.536603525045378
probability of observing that point if it came from cluster 0 =  0.2974823402921008
probability of observing that point if it came from cluster 1 =  4.853657071416018e-06
point =  6.311341892466097
probability of observing that point if it came from cluster 0 =  0.14275353622653922
probability of observing that point if it came from cluster 1 =  0.013613972346525784
point =  9.432593980640974
probability of observing that point if it came from cluster 0 =  0.0005298207783994897
probability of observing that point if it came from cluster 1 =  0.2687349377712282
point =  3.0617015000551646
probability of observing that point if it came from cluster 0 =  0.14192825703136822
probability of observing that point if it came from cluster 1 =  2.1138211040921487e-10
point =

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [16]:
prob_c = [sum(prob_s0_x)/ len(prob_s0_x), sum(prob_s1_x)/ len(prob_s1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x) ]
var = [ sum([x[0] * (x[1] - mean[0]) ** 2 for x in zip(prob_s0_x, data)]) / sum(prob_s0_x) , sum([x[0] * (x[1] - mean[1]) ** 2 for x in zip(prob_s1_x, data)]) / sum(prob_s1_x) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.4,  P(S_2) = 0.6
mean_1 = 4.842305559014659,  mean_2 = 8.565939032357578
var_1 = 1.6830383832142546,  var_2 = 0.8586588427681854


The estimated proportions P(S1) and P(S2) are mostly similar to the initial estimates obtained from K-means 
(0.7 and 0.3) respectively. Var1 has increased slightly anx Var2 increased significantly. These updates bring the parameters closer to the true distribution. 

g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [20]:
updated_prob_s0_x = []
updated_prob_s1_x = []

for p in data:
    pdf_i = []
    
    for j in range(k):
        pdf_i.append(norm.pdf(p, mean[j], var[j]))

    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    updated_prob_s0_x.append(pdf_i[0] * prob_s[0] / prob_x)
    updated_prob_s1_x.append(pdf_i[1] * prob_s[1] / prob_x)

probs = zip(data, updated_prob_s0_x, updated_prob_s1_x)
for p in probs:
    print("Data point:", p[0])
    print("Probability of coming from S_1:", p[1])
    print("Probability of coming from S_2:", p[2])
    print()

Data point: 8.772126951503878
Prob belonging to S_1: 0.022408980134235118
Prob belonging to S_2: 0.9775910198657649

Data point: 4.536603525045378
Prob belonging to S_1: 0.9999505878017666
Prob belonging to S_2: 4.9412198233459847e-05

Data point: 6.311341892466097
Prob belonging to S_1: 0.8795123134027281
Prob belonging to S_2: 0.12048768659727202

Data point: 9.432593980640974
Prob belonging to S_1: 0.013540998526618726
Prob belonging to S_2: 0.9864590014733813

Data point: 3.0617015000551646
Prob belonging to S_1: 0.9999999938554471
Prob belonging to S_2: 6.144552911589368e-09

Data point: 4.849106480214305
Prob belonging to S_1: 0.999749084289853
Prob belonging to S_2: 0.00025091571014694135

Data point: 8.041722539493131
Prob belonging to S_1: 0.06303551886281616
Prob belonging to S_2: 0.9369644811371838

Data point: 9.78287866474352
Prob belonging to S_1: 0.01233690857536605
Prob belonging to S_2: 0.9876630914246339

Data point: 8.13470102372501
Prob belonging to S_1: 0.053873560

Some data points moved closer to a binary characterization of which cluster they belong to. For example, Data point: 4.849106480214305 has an updated probability of 
0.999749084289853 belonging to S_1. However, others trend closer to 50/50 like Data point: 7.057658106142439. 

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [22]:
hard_assignments_labels = []

for p in zip(updated_prob_s0_x, updated_prob_s1_x):
    if p[0] > p[1]:
        hard_assignments_labels.append("Cluster 0 (S_1)")  # Assign to cluster 0 (S_1)
    else:
        hard_assignments_labels.append("Cluster 1 (S_2)")  # Assign to cluster 1 (S_2)

print("Hard assignments:")
for i, label in enumerate(hard_assignments_labels):
    print(f"Data point {i}: {label}")

Hard assignments:
Data point 0: Cluster 1 (S_2)
Data point 1: Cluster 0 (S_1)
Data point 2: Cluster 0 (S_1)
Data point 3: Cluster 1 (S_2)
Data point 4: Cluster 0 (S_1)
Data point 5: Cluster 0 (S_1)
Data point 6: Cluster 1 (S_2)
Data point 7: Cluster 1 (S_2)
Data point 8: Cluster 1 (S_2)
Data point 9: Cluster 1 (S_2)
